# Convex Optimization: Application to Energy Markets

**Prerequisites**

- Convex Optimization: Linear Programming
- 2-day Energy Markets in US


**Outcomes**

- See how to map real-world financial constraints into linear program
- Solve a 2-day portfolio optimization problem using linear programming
- Understand the issue of "corner-solutions" in linear programming

In [ ]:
# uncomment the line below if you do not have cvxpy installed
# %pip install cvxpy

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import scipy.sparse as sparse
import cvxpy as cp

## The Problem: Constrained Portfolio Optimization

Chase and Spencer were given the following task in a private sector engagement we had with a client:

We were asked to construct a profit maximizing portfolio of virtual bids, subject to the following risk constraints:

- No more than 500 units per day
- No more than 5 units long or short in any single hour
- Hourly net neutral: number of longs + number of shorts = 0

In an early iteration of the final solution, we set up the portoflio choice problem as a linear program

For the objective function we chose some approximation for the expected value of entering a long or short position

The risk limits were encoded as inequality constraints in the inequality $A_{\text{ub}} x \le b_{\text{ub}}$ and equation $A_{\text{eq}} x = b_{\text{eq}}$

In this lecture we will work through the details of how we accomplished this

## Data Preparation

The file `spp_prices_sample.parquet` contains hourly prices for a subset of non-correlated nodes from 2020-01-01 to 2020-10-31

Our goal today will be to use the data in this DataFrame to backtest a trading algorithm based on our linear programs for the month of October 2020

To do this we will pretend that for the date October 1, 2020 we only have access to the data through the end of September 2020. 

Then for, say, October 10th we will assume we only observe data up until October 9th and so on

In [ ]:
df_full = (
    pd.read_parquet("spp_prices_subset.parquet")
    .reset_index()
    .assign(dt=lambda x: x["pricedate"] + pd.Timedelta(hours=1) * (x["hour"] - 1))
    .set_index(["pricedate", "hour", "node_id"])
)
inds = [0, 1, 2, -3, -2, -1]
df_full.iloc[inds, :]

In [ ]:
n_nodes = df_full.index.get_level_values("node_id").nunique()
n_nodes

## Choosing the State

We'll stick to the mantra of "choosing the state is an art" and describe a key insight that allowed us to express this problem as a linear program

Our goal is to choose a portfolio that is *balanced* hour by hour -- meaning we buy and sell an equal number of units each hour 

A linear program always has the constraint that the choice variable $x$ be non-negative

In order to allow the linear program to enter a short position (buy a neagative amount in the day-ahead market),  we have two entries for each node and hour in our choice vector $x$: one for buying day-ahead and one for selling day-ahead

Thus, our choice vector will be length `n_nodes * n_hours * 2`, which for our example is 

In [ ]:
n_hours = 24
n_choice = n_nodes * n_hours * 2
n_choice

## Objective function

For the objective function, we need to capture the expected value of entering into each position

Because we are constrained to be net-zero each hour, the contribution of the marginal cost of energy will be zero

We'll focus on the cost of congestion and loss

Define the quantity:

$$rtda_{i,t} \equiv rtmcc_{i,t} + rtmcl_{i,t} - damcc_{i,t} - damcl_{i,t},$$

Which represents the spread between real time and day ahead congestion and loss at node $i$ in hour $t$

Let $x^+_{i,t}$ be the number of units long at node $i$ in hour $t$

The payoff for entering this position is equal to $$rtda_{i,t} \cdot x^+_{i,t}$$

Similarly let $x^-_{i,t}$ represent units short at node $i$ and hour $t$ with profits equal to $$-rtda_{i,t}\cdot x^-_{i,t}$$



We need to approximate this payoff in order to form the vector $c$ in the objective function of our linear program

We will first compute the quantity $rtda_{i,t}$ for all rows in our training dataset `df_full`

Then we will use the 30-day-trailling moving average of the hour-by-hour payoffs for each node...

In [ ]:
df_full["rtda"] = df_full.eval("rtmcc + rtmcl - damcc - damcl")
df_full["rtda_rolling_30d"] = (
    df_full
    .reset_index()
    .groupby(["node_id", "hour"])
    [["pricedate", "rtda"]]
    .apply(lambda x: x.set_index("pricedate").rolling("30d").mean().shift())
    .swaplevel("pricedate", "node_id")
    .sort_index()
)

# Notes: 

- We apply a `shift` after doing the 30 day moving average because when submitting day-ahead bids for day `t` we don't know all hours of day `t` real time prices. We only have full observation for hours in day `t-1`
- We `swaplevel` to let pandas align data for us using the index of `df_full` and the new 30-day column
- The `rtda_rolling_30d` for a particular date will be the vector $c$ associated with a short position across all nodes an hours for that date

We will define $x^+$ as a vector of all long positions, $x^-$ as the vector of all long positions and define

$$x \equiv \begin{bmatrix}x^+ \\ x^-\end{bmatrix},$$

As our vector of positions

Furthermore, we will impose that both $x^+$ and $x^-$ are lexographically sorted by hour and the node_id:

$$x^+ = \begin{bmatrix}x^+_{1,1} \\x^+_{2,1} \\ \vdots \\ x^+_{n_\text{nodes},1} \\ x^+_{1,2} \\ x^+_{2,2} \\ \vdots \\ x^+_{n_\text{nodes},2} \\ \vdots \\ x^+_{n_\text{nodes},24} \end{bmatrix}$$



We construct a series `c_full` that adheres to this ordering convention

In [ ]:
# note sign on the `rtda_rolling_30d`. It will correspond to 
# minus profits so we are ready for linprog to minimize negative profits
c_full = (
    pd.concat([
        df_full["rtda_rolling_30d"].rename("cplus"), 
        -df_full["rtda_rolling_30d"].rename("cminus")
    ], axis=1)
    .stack()
    .rename_axis(index=["pricedate", "hour", "node_id", "plus_minus"])
    .reorder_levels(["pricedate", "plus_minus", "hour", "node_id"])
    .sort_index()
)
c_full.tail()

## Constraints

We can map each of the risk constraints into rows of a constraint matrix $A$ and vector $b$

### Total volume $\le$ 500

We need a single row of $A$ equal to all ones. 

The corresponding element in $b$ will be 500

In [ ]:
A_volume = np.ones(n_choice).reshape((1, -1))
b_volume = np.array([500])

### 5 unit limit at each node and location

The second limit is that we can't have more than 5 units in any single position

This limit can be expressed by the identity matrix and a vector full of 5:

In [ ]:
A_nodal_volume = sparse.eye(n_choice)
b_nodal_volume = np.ones(n_choice) * 5

### Hourly net zero

Finally, we have a constraint that we must have a net zero position in each hour

This is the most difficult constraint for which to construct the rows of $A$ and $b$

Because we need the net position to be strictly equal to zero, we'll use the `A_eq` and `b_eq` arguments to `linprog` for expressing this risk limit

Consider a row of matrix $A$ specifying that we need to be net 0 for hour 10

We need to set this row entirely equal to 0, except for a 1 for all $x^+$ and a -1 for all $x^-$

This will cause us to compute $\sum_{i} x^+_{i,10} - x^-_{i,10}$

With a corresponding entry of $b$ of 0, this imposes the net-zero exposure constraint

Our strategy for computing these rows of $A$ will be to use the fact that the pandas index is already lexographically sorted by hour and node_id, and do some boolean computations on the `hour` level of the index to identify columns that pertain to each hour:

In [ ]:
A_eq = np.zeros((24, n_choice))

# get all hours for a specific day and subtract one to go from 0..23
c_hours = (c_full.loc["2020-10-01"].reset_index()["hour"] - 1).to_numpy()
scale_replacement = {"cminus": -1, "cplus": 1}
scale = (
    c_full.loc["2020-10-01"]
    .reset_index()
    ["plus_minus"]
    .replace(scale_replacement)
    .to_numpy()
)
for hour in range(24): 
    A_eq[hour, :] = (c_hours == hour) * scale
    
A_eq = sparse.csr_matrix(A_eq)
b_eq = np.zeros(24)

## Solving the Linear Program

In [ ]:
def solve_for_date(date):
    # extract c vector for chosen date
    c_date = c_full.loc[date]
    c = c_date.to_numpy()

    # build constraint matrices
    A_ub = sparse.vstack([A_volume, A_nodal_volume])
    b_ub = np.concatenate([b_volume, b_nodal_volume])

    # using cvxpy for performance gain.
    # alternative is to pass all the arrays constructed above
    # to scipy.optimize.linprog using the argument names we
    # assigned to these variables
    x = cp.Variable(n_choice)
    prob = cp.Problem(
        cp.Minimize(c @ x),
        [
            A_ub @ x <= b_ub,
            A_eq @ x == b_eq,
            x >= np.zeros(n_choice)
        ]
    )
    prob.solve()
    
    bids = (
        pd.Series(x.value, index=c_date.index, name="bid")
        .reset_index()
        .assign(pricedate=date)
    )
        
    # unpack constraints.
    
    # give them a helpful label so we can interpret them later
    nodal_volume_labels = [
        "node{node_id}_hour{hour}".format(**x) 
        for x in c_date.reset_index().to_dict(orient="records")
    ]
    cons_id = (
        ["volume"]*b_volume.size + 
        nodal_volume_labels + 
        [f"net_zero_hour_{i}" for i in range(1, 25)]
    )
    
    # package them up
    shadows = pd.Series(
        np.concatenate([x.dual_value for x in prob.constraints[:2]]),
        name="shadow_price"
    ).to_frame().assign(pricedate=date)
    shadows["cons_id"] = cons_id
    
    # return both
    return bids, shadows

In [ ]:
bid_shadows = []
for d in pd.date_range("2020-10-01", "2020-10-31"):
    print(f"solving for {d}")
    bid_shadows.append(solve_for_date(date=d))

bids, shadows = zip(*bid_shadows)
all_bids = pd.concat(bids)
all_shadows = pd.concat(shadows)

In [ ]:
output = (
    all_bids
    .reset_index()
    .assign(
        scale=lambda x: x["plus_minus"].replace(scale_replacement),
        bidvolume=lambda x: x.eval("scale*bid")
    )
    .merge(df_full["rtda"], on=["pricedate", "hour", "node_id"])
    .assign(
        gross_pl=lambda x: x.eval("- rtda * bidvolume")
    )
)

In [ ]:
# negative because we minimized negative profits
- output["gross_pl"].sum()

### Constraint Analysis

Let's look at which constraints were binding and how "expensive" these imposed limits were

In [ ]:
binding = all_shadows["shadow_price"] > 1e-2

In [ ]:
# show 20 most painful constraints
all_shadows.loc[binding, :].sort_values("shadow_price", ascending=False).head(20)

## Corner solution

The maximum position size we allowed in our algorithm was 5 units/node/hour

The total number of bids we allow in any day is 500

First, we verify that we did reach the 500 bids/day

In [ ]:
all_bids.groupby("pricedate").bid.sum()

Now, let's see how many non-zero nodes were bid on each day

If this is 100 then that means we bid the maximum 5 units on 100 nodes:

In [ ]:
(
    all_bids
    .groupby("pricedate")
    .apply(lambda x: x.query("bid > 1e-3").shape[0])
)

Notice that in each day, we bid 100 different positions, each at a volume of 5 units

Because these were our constraints, this is known as a corner solution

The linear program went all the way to the constraint for both the total bids per day and total bids per hour per node

This resulted in a portfolio that was less diverse than may be optimal if we were to take into account other risk factors

One very common risk factor that is used in portfolio optimization is the covariance of expected returns across positions

The full covariance structure is an inherently quadratic statistic and thus cannot be naturally included in our linear program

We would need to extend our framework to allow for a quadradtic objective function

In future lectures we will study quadratic programming and formulate a new objective that seeks to minimize a variance adjusted negative return

This is known as "modern portfolio theory" or Markowitz porftolio theory and was part of why Harry Markowitz earned a Nobel price in economics in 1990